## Esercizio 1.4

Scegliamo come verbo transitivo per questa esercitazione: `eat`

Il corpus `eat_corpus2.txt` presente nella cartella è stato costruito usando il sito di Sketch Engine.

Si è scelto di usare il verbo eat perchè su diversi tentativi è quello che ha portato ad ottenere un maggior numero di frasi rilevanti

Per l'esercitazione son stati costruiti altri corpus testabili: *eat_corpus, play_corpus, read_corpus*. Tuttavia si otteneva un numero di frasi rilevanti inferiore ai 1000. Per frasi rilevanti si considerano frasi da cui è possibile estrarre almeno una tripla Sog-Verbo-Ogg dove il verbo è *eat*
La differenza tra i due corpus *eat* è legata al numero di frasi, il secondo contiene oltre 450k frasi, anche se molte prive di senso nel complesso permette di ottenere molti dati in più.

Seguono una serie di metodi che saranno usati durante la fase di processing delle frasi, nella seguente sezione di codice si estraggono tutte le frasi dal corpus:

In [28]:
import os
import re

input_verb = 'eat'

str = open(os.getcwd()+'\\eat_corpus2.txt', 'r', encoding="utf8").read()
corpus_sents = re.findall(r'<p>(.*)</p>', str)

## Metodo: Extract SVO

Il seguente metodo, data una frase in input, restituisce una lista di triple Soggetto - Verbo - Oggetto trovate in essa.

Per questa fase si utilizza la libreria di spacy in fase di parsing e un modulo importato per l'estrazione delle triple.

Per via dell'ambiguità delle frasi possono essere individuati più soggetti e oggetti quindi il metodo restituirà una lista.

In [2]:
import spacy
from subject_object_extraction import findSVOs
nlp = spacy.load("en_core_web_sm")

def extract_svo(input_sentence):
    doc = nlp(input_sentence)
    return findSVOs(doc)

# Example 
print(extract_svo("The dog ate a disk"))
    

[('dog', 'ate', 'disk')]


## Word Sense Disambiguation

Costruaimo un metodo per la disambugiazione automatica di una parola in ingresso data la frase in cui è usata.

Per questa fase sarà utilizzato il disambiguatore delle librerie `nltk` che utilizza l'algoritmo di `Lesk`.

Dal momento che si tratta di un parsing superficiale, per non perdere l'informazione delle frasi dove il soggetto o oggetto compare come pronome, si è scelto di semplificare impostando quei termini come tipo Persona:<br>
In particolare usiamo una regex per identificarli nella parola in input e se si trova corrispondenza si restituisce il primo synset associato al termine "bambino" in quanto nella fase successiva, quando sarà estratto il supersense otterremo Persona.

In [20]:
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
prepositions = {'i', 'you', 'she', 'he', 'we', 'they'}

def select_disambiguated_sense(word, sentence):
    # Questo try serve perchè essendo un corpus generato automaticamente con grandi quantità di frasi possono capitarne
    # alcune con caratteri particolari che generano errori con le regex (in particolare le parentesi () [])
    #try:
    if word.lower() in prepositions: #re.search(rf'\s{word}\s', prepositions, re.IGNORECASE):
        return wn.synsets('kid')[0]
    #except Exception as e:
        #return None
    return lesk(sentence, word, 'n')

# Example
print(select_disambiguated_sense('apple', 'The dog eat an apple'))
print(select_disambiguated_sense('i', 'i eat an apple'))

Synset('apple.n.02')
Synset('child.n.01')


## Super sense extractor
Il seguente metodo dati input dei synset di WordNet restituisce i super sensi tramite la chiamata `lexname()` sul synset in input


In [5]:
def supersense(synset):
    return [s.lexname() for s in synset] if isinstance(synset, list) else synset.lexname()


# Example
print(supersense(wn.synsets('apple')))
print(supersense(wn.synsets('kid')[0]))

['noun.food', 'noun.plant']
noun.person


## The Big Boy

Ora che abbiamo definito tutti i metodi necessari per la fase centrale costruiamo il principale.
    
Per ogni tripla SVO ottenuta mediante la chiamata `extract_svo`:
        
    Memoriziamo in variabili il lemma dei rispettivi Soggetto e Oggetto e lo stemma per il verbo. E' stato scelto di usare lo stemma per il verbo per trovare maggiori confronti ed evitare di perdere frasi rilevanti.

    Disambiguiamo soggetto e oggetto. Nel caso in cui ottenessimo un risultato vuoto, quando WordNet non contiene un senso per la parola passata in input, useremmo il comando `continue` per saltare questa tripla in quanto non rilevante.
        <br>Discorso analogo per le triple con un verbo diverso da `eat`

    Una volta disambiguato estriamo i supersensi e aggiungiamo la tupla alla lista dei risultati che sarà restituita in output al termine.

In [6]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
ps = PorterStemmer()
lm = WordNetLemmatizer()  
 

def big_boy(sentence):
    result_list = []
    
    for svo_triple in extract_svo(sentence):
        subj = lm.lemmatize(svo_triple[0]) 
        verb = ps.stem(svo_triple[1])
        obj = lm.lemmatize(svo_triple[2]) 
        
        disambiguated_subj = select_disambiguated_sense(subj, sentence)
        disambiguated_obj = select_disambiguated_sense(obj, sentence)
        
        if verb != input_verb: 
            #print("WRONG VERB",verb)
            continue
        if disambiguated_subj is None or disambiguated_obj is None:
            #print(sentence)
            #print("NONE SVO: ", svo_triple)
            continue
            
        sub_supersense = supersense(disambiguated_subj)
        obj_supersense = supersense(disambiguated_obj)
        
        result_list.append((sub_supersense, obj_supersense))
        
        #print(svo_triple, " -> ", (sub_supersense, obj_supersense))
    return result_list
    
# Example
big_boy("The kid eat an apple")

[('noun.substance', 'noun.plant')]

## Risultati

Per concludere aggreghiamo i risultati usando il metodo `Counter`sulla lista ordinando le tuple così da considerare insieme i casi *(a,b)* e *(b,a)*

Ottenuto il risultato dei cluster con le rispettive frequenze stampiamo il valore e la percentuale sul totale dei cluster trovati.

In [30]:
from collections import Counter
import pprint
result_list = []
usefull_sentence = 0

for sent in corpus_sents:
    result = big_boy(sent)
    # l'if serve per ignorare i casi in cui il metodo non trova cluster e restituisce la lista vuota
    if result:
        result_list += (result)
        usefull_sentence += 1


count = Counter(tuple(sorted(t)) for t in result_list)
print("Number of usefull sentence found on corpus: ", usefull_sentence)
print("Number of usefull triple found on corpus: ", len(result_list))
print("\nResult")
for c in count.most_common():
    print(f'{c[0]}   freq: {c[1]}    perc: ', c[1]/len(result_list)*100)



Number of usefull sentence found on corpus:  1638
Number of usefull triple found on corpus:  2185

Result
('noun.artifact', 'noun.person')   freq: 249    perc:  11.395881006864988
('noun.food', 'noun.person')   freq: 212    perc:  9.702517162471395
('noun.person', 'noun.person')   freq: 180    perc:  8.237986270022883
('noun.person', 'noun.plant')   freq: 179    perc:  8.192219679633867
('noun.animal', 'noun.person')   freq: 62    perc:  2.8375286041189933
('noun.animal', 'noun.artifact')   freq: 61    perc:  2.7917620137299775
('noun.group', 'noun.person')   freq: 60    perc:  2.745995423340961
('noun.cognition', 'noun.person')   freq: 59    perc:  2.700228832951945
('noun.animal', 'noun.plant')   freq: 52    perc:  2.379862700228833
('noun.location', 'noun.person')   freq: 44    perc:  2.013729977116705
('noun.group', 'noun.plant')   freq: 41    perc:  1.8764302059496567
('noun.animal', 'noun.food')   freq: 41    perc:  1.8764302059496567
('noun.cognition', 'noun.plant')   freq: 39  